# Preparing Quantum States Workbook

This workbook describes the solutions to the problems offered in the "Preparing Quantum States" kata. Since the tasks are offered as programming problems, the explanations also cover some elements of Qiskit that might be non-obvious for a first-time user.

In [1]:
from qiskit import QuantumCircuit, QuantumRegister

## Exercise 1. Prepare superposition of all two-qubit basis vectors

You know that the Hadamard gate maps the basis state $\ket{0}$ to $\frac{1}{\sqrt2}(\ket{0} + \ket{1})$, so it's a logical starting point for solving this problem. 

Next, you see that the final state has a $\frac{1}{2}$ term hinting that you might be applying two operations involving a $\frac{1}{\sqrt{2}}$ term. 

Now, how do you get the $\ket{00} + \ket{01} + \ket{10} + \ket{11}$ expression? Let's see what multiplying the expression $\ket{0} + \ket{1}$ by itself looks like:

$$(\ket{0} + \ket{1}) \otimes (\ket{0} + \ket{1}) = \ket{0}\ket{0} + \ket{0}\ket{1} + \ket{1}\ket{0} + \ket{1}\ket{1}$$

Thus, applying the Hadamard gate to each qubit separately will deliver the desired final result:

$$H\ket{0} \otimes H\ket{0} = \frac{1}{\sqrt2} (\ket{0} + \ket{1}) \otimes \frac{1}{\sqrt2}(\ket{0} + \ket{1})
= \frac{1}{2} (\ket{00} + \ket{01} + \ket{10} + \ket{11})$$

In Qiskit, you can apply a gate to each qubit of the register by simply passing the entire register as the argument to the gate.

In [ ]:
def prepare_all_two_qubits(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.h(qr)

## Exercise 2. Prepare superposition of all two-qubit basis vectors with a phase flip on |11⟩

Here you start with the end state of the previous task $\frac{1}{2} (\ket{00} + \ket{01} + \ket{10} + \ket{11})$. Looking at the desired state, the phase of the $\ket{11}$ state is flipped ($+$ changed to a $-$).

A regular phase flip on one qubit can be done using a $Z$ gate:
$$\begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}$$
This gate will perform a phase flip only on the $\ket{1}$ state:

$$Z(\alpha\ket{0} + \beta\ket{1}) = \alpha\ket{0} - \beta\ket{1}$$

In this case, you only want to flip the phase of the $\ket{11}$ state and not the $\ket{01}$ state. To accomplish this, you can use a controlled $Z$ gate; this will make sure that the $Z$ gate is only applied if the control bit is in the $\ket{1}$ state, and the $\ket{01}$ state won't change.

In [ ]:
def prepare_all_two_qubits_phase_flip(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.h(qr)
    circ.cz(qr[0], qr[1])

## Exercise 3. Prepare superposition of all two-qubit basis vectors with complex phases

Let's approach the problem from the desired end result, and see if you can factor any expressions out of the target state:

$$
\ket{00} + i\ket{01} - \ket{10} - i\ket{11}
= \big(\ket{0} - \ket{1}\big) \ket{0} + \big(\ket{0} - \ket{1}\big) i\ket{1}
= \big(\ket{0} - \ket{1}\big) \otimes \big(\ket{0} + i\ket{1}\big)
$$

The fact that you were able to factor out the state into a tensor product of two terms means the state is separable.
This means that you need to prepare the first qubit in the state $\tfrac1{\sqrt2}(\ket{0} - \ket{1})$ and the second qubit - in the state $\tfrac1{\sqrt2}(\ket{0} + i\ket{1})$.

You've already seen that applying a Hadamard gate to a qubit in the $\ket{0}$ state allows you to prepare the state $\tfrac1{\sqrt2}(\ket{0} + \ket{1})$.

Next, let's take a look at the basic gates, in particular the Pauli Z gate:

$$Z = \begin{bmatrix} 1 & 0 \\ 0 & -1 \end{bmatrix}$$

If it's applied to the state $\frac{1}{\sqrt2} \big(\ket{0} + \ket{1}\big)$, it'll leave the basis state $\ket{0}$ unchanged and will map $\ket{1}$ to $-\ket{1}$. Thus, 

$$Z\tfrac{1}{\sqrt2} \big(\ket{0} + \ket{1}\big) = \tfrac{1}{\sqrt2} \big(\ket{0} - \ket{1}\big)$$

So the $Z$ gate is the answer to the question of how to do the first transformation. 

Looking for another gate to address the second transformation, you find the $S$ gate:

$$S = \begin{bmatrix} 1 & 0 \\ 0 & i \end{bmatrix}$$

If it's applied to the state $\frac{1}{\sqrt2} \big(\ket{0} + \ket{1}\big)$, it'll leave the basis state $\ket{0}$ unchanged and will map $\ket{1}$ to $i\ket{1}$. Thus, 

$$S\tfrac{1}{\sqrt2} \big(\ket{0} + \ket{1}\big) = \tfrac{1}{\sqrt2} \big(\ket{0} + i\ket{1}\big)$$

So the $S$ gate now answers the question of how to do the second transformation.

To summarize, the state you need to prepare can be represented as follows:
$$ZH\ket{0} \otimes SH\ket{0}$$

Remember that the gates have to be applied in reverse order compared to the mathematical notation: the gate closest to the ket symbol is applied first.

In [ ]:
def prepare_all_two_qubits_complex_phases(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.h(qr)
    circ.z(qr[0])
    circ.s(qr[1])

## Exercise 4. Prepare Bell state

The first thing you notice is that, unlike in the previous task, you can't represent this state as a tensor product of two individual qubit states - this goal state is **not** separable. 

> How can you see this? Let's assume that this state can be represented as a tensor product of two qubit states: 
>
> $$\ket{\psi_1} \otimes \ket{\psi_2} = (\alpha_1\ket{0} + \beta_1\ket{1}) \otimes (\alpha_2\ket{0} + \beta_2\ket{1}) = \alpha_1\alpha_2\ket{00} + \alpha_1\beta_2\ket{01} + \beta_1\alpha_2\ket{10} + \beta_1\beta_2\ket{11}$$
>
>In order for this state to be equal to $\frac{1}{\sqrt2}\big(\ket{00} + \ket{11}\big)$, you need to have $\alpha_1\alpha_2 = \beta_1\beta_2 = \frac{1}{\sqrt2}$ and at the same time $\alpha_1\beta_2 = \beta_1\alpha_2 = 0$, which is impossible.
>
>This is the phenomena called **entanglement**, in which the states of the qubits are linked together and can't be considered individually.  

First, notice that you should end with a superposition of two of the four computational basis for two qubits: $\ket{00}, \ket{01}, \ket{10}, \ket{11}$.

This gives you a hint that you should start by preparing a superposition on at least one of the qubits. Let’s try creating a superposition on the first qubit with a Hadamard gate: 

$$H\ket{0} \otimes \ket{0} = \tfrac{1}{\sqrt2} (\ket{0} + \ket{1}) \otimes \ket{0} = \tfrac{1}{\sqrt2} (\ket{00} + \ket{10})$$

Well, you got pretty close, except you need to transform the $\ket{10}$ state to $\ket{11}$.
How can you do this? 

You can take advantage of controlled gates, specifically the [controlled NOT gate](https://en.wikipedia.org/wiki/Controlled_NOT_gate), also referred to as CNOT. This gate acts on two qubits, hence it's represented as a $4 \times 4$ unitary matrix. The CNOT gate changes the target qubit from state $\ket{0}$ to $\ket{1}$ and vice versa when the control qubit is $\ket{1}$ and does nothing to the target qubit when the control qubit is $\ket{0}$. The control qubit always remains unchanged. 

$$\text{CNOT} = \begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix}$$

If you apply the CNOT gate to the state $\frac{1}{\sqrt2} (\ket{00} + \ket{10})$, taking the first qubit as the control and the second one as target, you'll get exactly the desired goal state. 
 
Steps required to reach goal state:
1. Apply a Hadamard gate to the first qubit.
2. Applying a CNOT gate with first qubit as control and second qubit as target.

In matrix representation, you can represent this operation as a product of two $4 \times 4$ matrices, with the matrix corresponding to the first step being the tensor product of a Hadamard gate on the first qubit and identity gate on the second qubit.

$$H \otimes I = \frac{1}{\sqrt2} \begin{bmatrix} 1 & 1  \\ 1 & -1 \end{bmatrix} \otimes \begin{bmatrix} 1 & 0  \\ 0 & 1 \end{bmatrix} = 
\frac{1}{\sqrt2}\begin{bmatrix} 1 & 0 & 1 & 0 \\ 0 & 1 & 0 & 1 \\ 1 & 0 & -1 & 0 \\ 0 & 1 & 0 & -1 \end{bmatrix}$$

$$\begin{bmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \end{bmatrix}
\cdot 
\frac{1}{\sqrt2} \begin{bmatrix} 1 & 0 & 1 & 0 \\ 0 & 1 & 0 & 1 \\ 1 & 0 & -1 & 0 \\ 0 & 1 & 0 & -1 \end{bmatrix}
\cdot
\begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \end{bmatrix}
= \frac{1}{\sqrt2} \begin{bmatrix} 1 & 0 & 1 & 0 \\ 0 & 1 & 0 & 1 \\ 0 & 1 & 0 & -1 \\ 1 & 0 & -1 & 0 \end{bmatrix}
\cdot
\begin{bmatrix} 1 \\ 0 \\ 0 \\ 0 \end{bmatrix}
= \frac{1}{\sqrt2} \begin{bmatrix} 1 \\ 0 \\ 0 \\ 1 \end{bmatrix}
$$

In [ ]:
def prepare_bell_state(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.h(qr[0])
    circ.cx(qr[0], qr[1])

## Exercise 5. Prepare any Bell state

> The [Bell states](https://en.wikipedia.org/wiki/Bell_state) form an orthonormal basis in the 4-dimensional space that describes the states of a 2-qubit system. 
You can check that the norm of each of these states is 1, and their inner product of each pair of states is 0.

The goal is to transform the $\ket{00}$ basis state into one of the Bell basis states, depending on the value of `index` given as an input.

This exercise provides two solutions, one of which is based on the previous exercise, and the second one helps you understand the unitary transformation that converts the computational basis into the Bell basis.

### Solution 1

Let's use the first Bell state you prepared in the previous exercise, $\tfrac{1}{\sqrt2} (\ket{00} + \ket{11})$, and transform it according to the value of `index`.
What transformation do you need to apply to get to the final state?

* If `index = 0`, you do nothing - the prepared state is already $\ket{\Phi^{+}}$.

* If `index = 1`, you need to add a relative phase of $-1$ to the $\ket{11}$ term. Remember that $Z$ gate does exactly that with a qubit:
  
  $$Z(H\ket{0}) \otimes \ket{0} = \tfrac{1}{\sqrt2} \big(\ket{0} - \ket{1}\big) \otimes \ket{0} = \tfrac{1}{\sqrt2} \big(\ket{00} - \ket{10}\big)$$
  
  If you now apply the $CNOT$ as before, you'll have:

  $$\tfrac{1}{\sqrt2} \big(\ket{00} - \ket{10}\big) \Longrightarrow \tfrac{1}{\sqrt2} \big(\ket{00} - \ket{11}\big) = \ket{\Phi^{-}}$$

* If `index = 2`, you need to change the second qubit in both $\ket{00}$ and $\ket{11}$ terms, which can be done applying an $X$ gate:
  
  $$H\ket{0} \otimes X\ket{0} = H\ket{0} \otimes \ket{1} = \tfrac{1}{\sqrt2} \big(\ket{0} + \ket{1}\big) \otimes \ket{1} = \tfrac{1}{\sqrt2} \big(\ket{01} + \ket{11}\big)$$
  
  If you now apply the $CNOT$ as before, you'll have:
  
  $$\tfrac{1}{\sqrt2} \big(\ket{01} + \ket{11}\big) \Longrightarrow \tfrac{1}{\sqrt2} \big(\ket{01} + \ket{10}\big) = \ket{\Psi^{+}}$$

* If `index = 3`, you use the same logic to realize that you need to apply both the $Z$ and $X$ corrections to get $\ket{\Psi^{-}}$ state.

The final sequence of steps is as follows:
1. Apply the $H$ gate to the first qubit. 
2. Apply the $Z$ gate to the first qubit if `index == 1` or `index == 3`.
3. Apply the $X$ gate to the second qubit if `index == 2` or `index == 3`.
4. Apply the $CNOT$ gate with the first qubit as control and the second qubit as target.

In [ ]:
def prepare_any_bell_state(circ: QuantumCircuit, qr: QuantumRegister, index: int) -> None:
    circ.h(qr[0])
    circ.cx(qr[0], qr[1])
    if index % 2 == 1:
        circ.z(qr[0])
    if index > 1:
        circ.x(qr[1])

### Solution 2

Let's take a closer look at the unitary transformation $\text{CNOT}\cdot(H \otimes I)$ discussed in the previous task.

$$\frac{1}{\sqrt2} \begin{bmatrix} 1 & 0 & 1 & 0 \\ 0 & 1 & 0 & 1 \\ 0 & 1 & 0 & -1 \\ 1 & 0 & -1 & 0 \\ \ket{\Phi^{+}} & \ket{\Psi^{+}} & \ket{\Phi^{-}} & \ket{\Psi^{-}} \end{bmatrix}$$


Notice that each of the columns in the unitary matrix corresponds to one of the Bell states.
This unitary transformation transforms the computational basis into the Bell basis, which is exactly what the task asks you to do.

You see that this transformation converts $\ket{00}$ into the first Bell state, $\ket{01}$ into the second Bell state, etc. 
You just need to make sure you set the qubits to the correct state before applying this transformation, using $X$ gates to change the initial $\ket{0}$ states to $\ket{1}$ if needed. 

In [ ]:
def prepare_any_bell_state(circ: QuantumCircuit, qr: QuantumRegister, index: int) -> None:
    if index % 2 == 1:
        circ.x(qr[0])
    if index > 1:
        circ.x(qr[1])
    circ.h(qr[0])
    circ.cx(qr[0], qr[1])

## Exercise 6. Prepare Greenberger-Horne-Zellinger (GHZ) state

The single-qubit GHZ state is the plus state $\frac{1}{\sqrt{2}} \big (\ket{0} + \ket{1}\big)$ that you've discussed in the first task. As a reminder, that state is prepared by applying a Hadamard gate.

The 2-qubit GHZ state is the Bell state $\frac{1}{\sqrt{2}} \big (\ket{00} + \ket{11}\big)$ that we've discussed in the two previous tasks.

The next one is the 3-qubit GHZ state:
$$\ket{GHZ} = \tfrac{1}{\sqrt{2}} \big (\ket{000} + \ket{111}\big)$$

Let's use the 2-qubit state as a building block to construct the state for 3 qubits. First, let's add a third qubit to the above state (on the right from the first two qubits).
Comparing this state with the desired end state, you see that they differ only in the third (rightmost) qubit:

$$\ket{\Phi^+} \ket{0} = \tfrac{1}{\sqrt{2}} \big (\ket{000} + \ket{11\textbf{0}}\big)$$
$$\ket{GHZ} = \tfrac{1}{\sqrt{2}} \big (\ket{000} + \ket{11\textbf{1}}\big)$$

Applying a controlled $X$ gate using the first (leftmost) qubit as the control bit and the third (rightmost) qubit as the target qubit allows you to fix this difference.

Thus, you can come to the general solution: apply Hadamard gate to the first qubit and do a series of CNOT gates with the first qubit as control and each of the other qubits as targets.

In [ ]:
def prepare_ghz_state(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.h(qr[0])
    for ind in range(1, qr.size):
        circ.cx(qr[0], qr[ind])

## Exercise 7. Prepare superposition of all N-qubit basis vectors

As you've seen in exercise 1, to prepare a superposition of all basis vectors on 2 qubits you need to apply a Hadamard gate to each of the qubits.

It seems that the solution for the general case might be to apply a Hadamard gate to every qubit as well. Let's check the first few examples:

$$\begin{align*}
   H\ket{0} &= \tfrac{1}{\sqrt2}\big(\ket{0} + \ket{1}\big) \\
   H\ket{0} \otimes H\ket{0} &= \tfrac{1}{\sqrt2} \big(\ket{0} + \ket{1}\big) \otimes \tfrac{1}{\sqrt2} \big(\ket{0} + \ket{1}\big) \\ 
               &= \tfrac{1}{\sqrt{2^2}}\big(\ket{00} + \ket{01}+ \ket{10}+ \ket{11}\big) \\
   H\ket{0} \otimes H\ket{0} \otimes H\ket{0} &= \tfrac{1}{\sqrt{2^2}}\big(\ket{00} + \ket{01}+ \ket{10}+ \ket{11}\big) \otimes \tfrac{1}{\sqrt2}\big(\ket{0} + \ket{1}\big) \\
               &= \tfrac{1}{\sqrt{2^3}}\big(\ket{000} + \ket{001} + \ket{010}+ \ket{100}+ \ket{110} + \ket{101}+ \ket{011}+ \ket{111}\big)
\end{align*}$$

For the $N$-qubit case, we can express it as a product of the $N-1$-qubit case and the single-qubit case:

$$\begin{align*}
H\ket{0} \otimes \dots \otimes H\ket{0}
               &= \tfrac{1}{\sqrt{2^{N-1}}} \big( \ket{0 \cdots 0}_{N-1} + \cdots + \ket{1 \cdots 1}_{N-1} \big) \otimes \tfrac{1}{\sqrt2}\big(\ket{0} + \ket{1}\big) = \\
               &= \tfrac{1}{\sqrt{2^N}} \big( \ket{0 \cdots 0}_N + \cdots + \ket{1 \cdots 1}_N \big) 
\end{align*}$$

Thus, the solution requires you to iterate over the qubit array and to apply the Hadamard gate to each element.

In [ ]:
def prepare_all_n_qubits(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.h(qr)

## Exercise 8. Prepare superposition of all even or all odd numbers

Let’s look at some examples of basis states to illustrate the binary numbering system. 

The 4 basis states on $N = 2$ qubits can be split in two columns, where the left column represents the basis states that form the required superposition state for `even = True` and the right column - the basis states that form the required superposition state for `even = False`.

| even | odd |
| ---- | --- |
| 0**0** | 1**0** |
| 0**1** | 1**1** |
 
If you do the same basis state split for $N = 3$ qubits, the pattern becomes more obvious.

| even | odd |
| ---- | --- |
| 0**00** | 1**00** |
| 0**10** | 1**10** |
| 0**01** | 1**01** |
| 0**11** | 1**11** |


The two rightmost qubits go through all possible basis states for `even = True` and for `even = False`, and the leftmost qubit stays in the $\ket{0}$ state for `even = True` and in the $\ket{1}$ state for `even = False`. 

A quick sanity check for $N = 4$ qubits re-confirms the pattern.

| even | odd |
| ---- | --- |
| 0**000** | 1**000** |
| 0**100** | 1**100** |
| 0**010** | 1**010** |
| 0**110** | 1**110** |
| 0**001** | 1**001** |
| 0**101** | 1**101** |
| 0**011** | 1**011** |
| 0**111** | 1**111** |

Again, the three rightmost qubits go through all possible basis states in both columns, and the leftmost qubit stays in the same state in each column. 

The solution is to put all qubits except the leftmost one into an equal superposition (similar to what you did in the previous task) and to set the leftmost qubit to $\ket{0}$ or $\ket{1}$ depending on the `even` flag, using the $X$ operator to convert $\ket{0}$ to $\ket{1}$ if `even = False`.

In [ ]:
def prepare_even_odd_numbers(circ: QuantumCircuit, qr: QuantumRegister, even: bool) -> None:
    if not even:
        circ.x(qr[0])
    if qr.size > 1:
        circ.h(qr[1:])

## Exercise 9. Prepare superposition of all-zero and given bit string

> A common strategy for preparing a superposition state in a qubit register is using an auxiliary qubit (or several, for more complicated states). The auxiliary qubit can be put into a superposition state through the usual means of applying a Hadamard gate (or a rotation about the $Y$ axis for an uneven superposition). 
> Then the basis states of the desired superposition are prepared individually based on the auxiliary qubit state by using it as the control qubit for a CNOT gate. One of the basis states will be prepared controlled on the $\ket{0}$ component of the auxiliary state, and the other - controlled on the $\ket{1}$ component. 
> Finally, you have to return the auxiliary qubit to the $\ket{0}$ state by uncomputing it, that is, by using the basis state prepared from the $\ket{1}$ component as the control qubits for a $CNOT$ gate with the auxiliary qubit as the target. 
>
> You will see examples of using this approach in later exercises of this kata. However, this exercise has a simpler solution. 
> Instead of allocating a new qubit to use as the auxiliary, you can use the first qubit in the register for this purpose, because you're guaranteed that the first bit in the two basis vectors that comprise the required superposition is different.
> This saves you the need to allocate a new qubit and lets us skip the uncomputing step, as the qubit acting as the control for the next preparation steps is part of the desired result.

Consider the earlier exercises in this kata that asked to prepare Bell states and GHZ state. The structure of the superposition state in this task is a more general case of those scenarios: all of them ask to prepare an equal superposition of two different basis states.

The first step of the solution is the same as in those exercises: put the first qubit in the register into an equal superposition of $\ket{0}$ and $\ket{1}$ using the $H$ gate to get the following state:

$$\tfrac{1}{\sqrt2} (\ket{0} + \ket{1}) \otimes \ket{0 \dots 0} = \tfrac{1}{\sqrt2} (\ket{00 \dots 0} + \ket{10 \dots 0})$$

The first term of the superposition already matches the desired state, so you need to fix the second term.
To do that, you'll walk through the remaining qubits in the register, checking if the bit in the corresponding position of the bit string `bits` is `True`. 
If it is, that qubit's state needs to be adjusted from $0$ to $1$ in the second term of the superposition (and left unchanged in the first term). 
You can do this change using the CNOT gate with the first qubit as the control and the current qubit as the target.
When you have finished walking through the register like this, the register will be in the desired superposition.

In [ ]:
def prepare_zero_and_bitstring(circ: QuantumCircuit, qr: QuantumRegister, bits: list[bool]) -> None:
    circ.h(qr[0])
    for ind in range(1, qr.size):
        if bits[ind]:
            circ.cx(qr[0], qr[ind])

## Exercise 10. Prepare superposition of two bit strings

The strategy of using an auxiliary qubit to control the preparation process described in the previous exercise can be applied to this task as well. 

You'll start by preparing the given auxiliary qubit in the $\tfrac{1}{\sqrt2} (\ket{0} + \ket{1})$ state using the $H$ gate. The overall state of the system will be 

$$\tfrac{1}{\sqrt2} (\ket{0} + \ket{1})_{aux} \otimes \ket{0 \dots 0}_{qr} = \tfrac{1}{\sqrt2} (\ket{0}_{aux} \otimes \ket{0 \dots 0}_{qr} + \ket{1}_{aux} \otimes \ket{0 \dots 0}_{qr})$$

At this point, you can prepare the two basis states of the target state separately, bit by bit, controlling the preparation of one of them on the $\ket{0}$ state of the auxiliary qubit and the preparation of the other one - on the $\ket{1}$ state. 
If a bit in one of the bit strings is `True`, you'll apply a controlled $X$ gate with the auxiliary qubit as control, the qubit in the corresponding position of the register as target, and control it on the $\ket{0}$ or the $\ket{1}$ state depending on which bit string you're considering at the moment. 
Such controlled gate can be implemented using the `cx` method with the `ctrl_state` argument set to `0` for `bits1` and to `1` for `bits2`.

After this the state of the system will be 
$$\tfrac{1}{\sqrt2} (\ket{0}_{aux} \otimes \ket{bits_1}_{qr} + \ket{1}_{aux} \otimes \ket{bits_2}_{qr})$$

Finally, you need to uncompute the auxiliary qubit. You can do this by finding the first index at which the bit strings `bits1` and `bits2` differ and using the corresponding qubit as the control for an X gate with the auxiliary qubit as the target. You'll need to make sure that this gate will affect only the $\ket{1}_{aux} \otimes \ket{bits_2}_{qr}$ term. You can do this using the `ctrl_state` argument of the `cx` method to specify the control pattern to be $1$ if the corresponding bit of `bits2` is `True`, and $0$ if it is `False`. This will bring the system to its final state:

$$\ket{0}_{aux} \otimes \tfrac{1}{\sqrt2} (\ket{bits_1} + \ket{bits_2})_{qr}$$

In [ ]:
def prepare_two_bitstrings(circ: QuantumCircuit, qr: QuantumRegister, aux: QuantumRegister, bits1: list[bool], bits2: list[bool]) -> None:
    circ.h(aux)
    for ind in range(qr.size):
        # Control bits1 on aux=|0⟩
        if bits1[ind]:
            circ.cx(aux, qr[ind], ctrl_state=0)
        # Control bits2 on aux=|1⟩
        if bits2[ind]:
            circ.cx(aux, qr[ind], ctrl_state=1)

    # Uncompute aux
    for ind in range(qr.size):
        if bits1[ind] != bits2[ind]:
            diff = ind
            break
    circ.cx(qr[diff], aux, ctrl_state=1 if bits2[diff] else 0)

## Exercise 11. Prepare superposition of four bit strings

You're going to use the same trick of using auxiliary qubits that you used in the previous exercise.

Since the desired superposition has 4 basis states with equal amplitudes, you're going to need two qubits to define a unique basis to control preparation of each of the basis states in the superposition.

You start by preparing the two auxiliary qubits in an equal superposition of all 2-qubit states by applying an $H$ gate to each of them:

$$\tfrac12 (\ket{00} + \ket{01} + \ket{10} + \ket{11})_{aux} \otimes \ket{0 \dots 0}_{qr}$$

Then, for each of the four given bit strings, you walk through it and prepare the matching basis state on the main register of qubits, using controlled $X$ gates with the corresponding basis state of the auxiliary qubits as control.

For example, when preparing the bit string `bits[0]`, you apply $X$ gates controlled on the basis state $\ket{00}$; when preparing the bit string `bits[1]`, you apply $X$ gates controlled on $\ket{10}$, and so on.

> You can choose an arbitrary matching of the 2-qubit basis states used as controls and the bit strings prepared on the main register.
> Since all amplitudes are the same, the result doesn't depend on which state controlled which bit string preparation.
> It can be convenient to use indices of the bit strings themselves as the `ctrl_state` argument.

After this, the system will be in the state

$$\tfrac12 (\ket{00}_{aux} \ket{bits_0}_{qr} + \ket{10}_{aux} \ket{bits_1}_{qr} + \ket{01}_{aux} \ket{bits_2}_{qr} + \ket{11}_{aux} \ket{bits_3}_{qr})$$

As the last step, you must uncompute the auxiliary qubits, that is, return them to the $\ket{00}$ state to unentangle them from the main register.

Same as you did in the previous exercise, you can do that using controlled $X$ gates with the quantum register as the control, one of the given bit strings as the control pattern, and each of the auxiliary qubits as the target.
You'll uncompute each of the auxiliary qubits separately, so one of the auxiliary qubits will be uncomputed with the `bits[1]` and `bits[3]` bit strings as controls, and the other - with the `bits[2]` and `bits[3]`.

> Note the use of `XGate()` to construct controlled $X$ gates with an arbitrary number of qubits as controls.

In [ ]:
def prepare_four_bitstrings(circ: QuantumCircuit, qr: QuantumRegister, aux: QuantumRegister, bits: list[list[bool]]) -> None:
    from qiskit.circuit.library.standard_gates import XGate

    n = len(qr)
    # Put two auxiliary qubits into equal superposition of 2-qubit basis states
    circ.h(aux)

    for ind in range(n):
        for bits_ind in range(4):
            # Control bits[ind] on aux=ind
            if bits[bits_ind][ind]:
                circ.append(XGate().control(2, ctrl_state=bits_ind), aux[:] + [qr[ind]])

    # Uncompute auxiliary qubits
    def bitstring_as_int(bits):
        return sum([2 ** i if bits[i] else 0 for i in range(n)])

    for bits_ind in range(1, 4):
        ctrl = bitstring_as_int(bits[bits_ind])
        if bits_ind % 2 == 1:
            circ.append(XGate().control(n, ctrl_state=ctrl), qr[:] + [aux[0]])
        if bits_ind // 2 == 1:
            circ.append(XGate().control(n, ctrl_state=ctrl), qr[:] + [aux[1]])

## Exercise 12. Prepare superposition of bit strings of the given parity

This exercise can have multiple solutions. The easiest one to implement in Qiskit is the one that prepares the superposition of all possible basis states on all qubits except the first one and using the first qubit to adjust the parity of the resulting state.

You start by preparing a superposition of all basis states with parity $0$. To achieve that, you loop through all the qubits except the first one and prepare all of them in equal superposition. In order to maintain the parity of the basis states involved, you use the first qubit and conditionally flip its state using a CNOT gate with each next qubit as the control, so that every time there is a $\ket{1}$ state in the chain, you flip the state of the first qubit to maintain an even number of $\ket{1}$ states in that basis state.

For example, after the first loop iteration, you get the state $\tfrac12(\ket{00} + \ket{11})$. After the second iteration, you get the state

$$\text{CNOT}_{2,0} \tfrac1{\sqrt2}(\ket{00} + \ket{11}) \otimes \frac1{\sqrt2}(\ket{0} + \ket{1}) = $$
$$= \text{CNOT}_{2,0} \tfrac12(\ket{000} + \ket{\textbf{0}0\textbf{1}} + \ket{110} + \ket{\textbf{1}1\textbf{1}}) = $$
$$= \tfrac12(\ket{000} + \ket{101} + \ket{110} + \ket{011})$$

After the loop you'll have a superposition of all possible basis states with even number of $1$s.
Then, if `parity` is equal to 1, and we want an odd number of $\ket{1}$s, you just flip the state of the first qubit again (or any qubit).

For example, if the input has 3 qubits, after the for loop you'll have a superposition of 4 basis states:

$$\tfrac12(\ket{000} + \ket{101} + \ket{110} + \ket{011})$$

If `parity = 0`, you're done, having even numbers of $\ket{1}$s. If `parity = 1`, you flip the state of the first qubit, getting the desired result:

$$\tfrac12(\ket{100} + \ket{001} + \ket{010} + \ket{111})$$



In [ ]:
def prepare_given_parity(circ: QuantumCircuit, qr: QuantumRegister, parity: int) -> None:
    for ind in range(1, qr.size):
        circ.h(qr[ind])
        circ.cx(qr[ind], qr[0])

    if parity == 1:
        circ.x(qr[0])

## Exercise 13. Prepare uneven superposition of three two-qubit basis vectors

You'll start the preparation by putting the first qubit in the state $\alpha\ket{0} + \beta\ket{1}$, where $\alpha$ and $\beta$ are the square roots of relative weights of all basis states which start with 0 and with 1, respectively. After that, you can adjust the amplitudes further using controlled gates with the first qubit as the control.

To get the values of $\alpha$ and $\beta$, you represent the goal state as $\frac{1}{\sqrt{2}} \big( \ket{0} \otimes \ket{0} + \ket{1} \otimes \frac{1}{\sqrt2}(\ket{0}+\ket{1}) \big)$.

You see that the relative weights of $\ket{0}$ and $\ket{1}$ states of the first qubit are both $\frac12$ (the squares of their amplitudes in the decomposition above). This means that you can do the first step by applying the $H$ gate to the first qubit, which gives the $\frac{1}{\sqrt{2}}\ket{00} + \frac{1}{\sqrt{2}}\ket{10}$ state. 

Now the first term of the state $\frac{1}{\sqrt2}\ket{00}$ matches that of the goal state, and you need to convert the second term $\frac{1}{\sqrt2}\ket{10}$ to $\ket{1} \otimes \frac{1}{\sqrt2}(\ket{0}+\ket{1})$.

To do this, you use the controlled $H$ gate, with the first qubit as control and the second qubit as target.
When it is applied to the current state, you get exactly the goal state:

$$\text{Controlled } H \cdot (\tfrac{1}{\sqrt{2}}\ket{00} + \tfrac{1}{\sqrt{2}}\ket{10}) = \tfrac{1}{\sqrt{2}}\ket{00} + \tfrac{1}{2}\ket{10} + \tfrac{1}{2}\ket{11}$$

In [ ]:
def prepare_uneven_two_qubit(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    circ.h(qr[0])
    circ.ch(qr[0], qr[1])

## Exercise 14. Prepare even superposition of three two-qubit basis vectors

To start, you'll try to find a decomposition of the target state that makes it easier to see how to prepare the state.  
Knowing that $\ket{+} = \frac{1}{\sqrt{2}}\big(\ket{0}+\ket{1}\big)$, you can represent the state as follows:

$$\tfrac{1}{\sqrt{3}} \big(\ket{00} + \ket{01} + \ket{10}\big) = \tfrac{\sqrt{2}}{\sqrt{3}}\ket{0} \otimes \ket{+} + \tfrac{1}{\sqrt{3}}\ket{1} \otimes \ket{0}$$

To prepare this state, you first want to prepare the first qubit in the state $\frac{\sqrt{2}}{\sqrt{3}}\ket{0} + \frac{1}{\sqrt{3}}\ket{1}$, while leaving the second qubit unchanged.

To do this, you can use a rotation gate $R_y$ which will perform the following transformation:

$$R_y\ket{0} = \cos\tfrac{\theta}{2}\ket{0} + \sin\tfrac{\theta}{2}\ket{1}$$

You need to find a value of $\theta$ which satisfies both: 
$$\cos\tfrac{\theta}{2} = \tfrac{\sqrt{2}}{\sqrt{3}} \text{ and } \sin\tfrac{\theta}{2} = \tfrac{1}{\sqrt{3}}$$

Solving the last equation for $\theta$ gives you $\frac{\theta}{2} = \arcsin\frac{1}{\sqrt{3}}$, or $\theta = 2 \arcsin\frac{1}{\sqrt{3}}$.

When you apply this to your first qubit, you'll get your desired intermediary state:
$$ R_y(2 \arcsin\tfrac{1}{\sqrt{3}})\ket{0} \otimes \ket{0} = 
\left(\tfrac{\sqrt{2}}{\sqrt{3}}\ket{0} + \tfrac{1}{\sqrt{3}}\ket{1} \right) \otimes \ket{0} = 
\tfrac{\sqrt{2}}{\sqrt{3}}\ket{0} \otimes \ket{0} + \tfrac{1}{\sqrt{3}}\ket{1} \otimes \ket{0}$$

Now, the second term of this state already matches your final goal, so you need to adjust the first term: 
prepare the $\ket{+}$ state on the second qubit only if the first qubit is in the $\ket{0}$ state. 
To do this, you apply a conditional $H$ gate to the second qubit if the first qubit is in the $\ket{0}$ state. This will give your desired state:

$$ \tfrac{\sqrt{2}}{\sqrt{3}}\ket{0} \otimes \ket{+} + \tfrac{1}{\sqrt{3}}\ket{1} \otimes \ket{0}$$

In [ ]:
def prepare_even_two_qubit(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    from math import asin, sqrt
    theta = asin(1 / sqrt(3))
    circ.ry(2 * theta, qr[0])
    circ.ch(qr[0], qr[1], ctrl_state=0)

## Exercise 15. Prepare even superposition of three two-qubit basis vectors with complex phases

To start, you'll prepare the $\frac{1}{\sqrt{3}} \big( \ket{00} + \ket{01} + \ket{10} \big)$ state using the solution to the previous exercise. To get to the final state, you need to add the relative phases to both $\ket{01}$ and $\ket{10}$ basis states without changing the $\ket{00}$ state.

First, you want to transform the $\ket{01}$ state to the $\omega \ket{01} = e^{2\pi i/3} \ket{01}$ state while not changing the other states.
Using the $R_1$ gate, you can change a qubit state from $\ket{1}$ to $e^{i\theta}\ket{1}$ without changing the $\ket{0}$ state.
Indeed, here is the effect of the $R_1$ gate on the $\ket{0}$ and the $\ket{1}$ states:

$$ R_1 \ket{0} = \begin{bmatrix} 1 & 0 \\ 0 & e^{i\theta} \end{bmatrix} \cdot \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \begin{bmatrix} 1 \\ 0 \end{bmatrix} = \ket{0} $$

$$ R_1 \ket{1} = \begin{bmatrix} 1 & 0 \\ 0 & e^{i\theta} \end{bmatrix} \cdot \begin{bmatrix} 0 \\ 1 \end{bmatrix} = \begin{bmatrix} 0 \\ e^{i\theta} \end{bmatrix} = e^{i\theta}\ket{1} $$

When you apply the $R_1$ gate to the second qubit, this will only affect the $\ket{01}$ term, which is exactly what you want. Knowing this, you just need the right value for $\theta$, which in this case is $\frac{2\pi}{3}$.

You use the same approach to change $\ket{10}$ term to $\omega^2 \ket{10}$. By applying the $R_1$ gate to the first qubit you'll only change the $\ket{10}$ term. In this case the right $\theta$ will be $\frac{4\pi}{3}$.

In [ ]:
def prepare_even_two_qubit_phases(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    from math import pi
    prepare_even_two_qubit(circ, qr)
    circ.p(4 * pi / 3, qr[0])
    circ.p(2 * pi / 3, qr[1])

## Exercise 16. Prepare Hardy state

**Step 1:** Start by putting the first qubit in the state $\alpha\ket{0} + \beta\ket{1}$, where $\alpha$ and $\beta$ are the square roots of relative weights of all basis states which start with 0 and with 1, respectively.

In case of Hardy state, two basis states start with 0: $\frac{1}{\sqrt{12}} \big (3\ket{00} + \ket{01}\big )$ and two basis states start with 1: $\frac{1}{\sqrt{12}} \big (\ket{10} + \ket{11}\big )$.
The relative weights of 0 and 1 are just the sums of squares of their amplitudes:
$\alpha^2 = \frac{9}{12} + \frac{1}{12} = \frac{10}{12}$ and $\beta^2 = \frac{1}{12} + \frac{1}{12} = \frac{2}{12}$, respectively.
So you'll need to put the first qubit in the state $\sqrt{\frac{10}{12}}\ket{0} + \sqrt{\frac{2}{12}}\ket{1}$ using the $R_y$ gate:

$$\ket{00} \overset{R_{y_1}}\rightarrow \big (\sqrt{\frac{10}{12}}\ket{0} + \sqrt{\frac{2}{12}}\ket{1} \big ) \otimes \ket{0} =: \ket{\psi_1}$$

Here $R_{y_1} := R_y(2\arccos \sqrt{\frac{10}{12}}) \otimes I$.

**Step 2:** Finish the preparation by putting the second qubit in the right state, applying controlled $R_y$ gates with the first qubit as the control.

To get the first two terms right, you need to convert the terms

$$\big (\sqrt{\frac{10}{12}}\ket{0} \big) \otimes \ket{0} \overset{R_{y_2}}\rightarrow \frac{1}{\sqrt{12}} \big (3\ket{00} + \ket{01} \big) \text{  (2.a) }$$
and
$$\big (\sqrt{\frac{2}{12}}\ket{1} \big) \otimes \ket{0} \overset{R_{y_3}}\rightarrow \frac{1}{\sqrt{12}} \big (\ket{10}+\ket{11} \big) \text{  (2.b) }$$

**Step 2.a:** The transformation
$$\big (\sqrt{\frac{10}{12}}\ket{0} \big) \otimes \ket{0} \overset{R_{y_2}}\rightarrow \frac{1}{\sqrt{12}} \big (3\ket{00} + \ket{01} \big)$$

is equivalent to the following transformation (to see this, just multiply both sides by $\sqrt{\frac{12}{10}}$):

$$\ket{0} \otimes \ket{0} \overset{R_{y_2}}\rightarrow \frac{1}{\sqrt{10}} \big (3\ket{00} + \ket{01} \big) = \ket{0} \otimes \frac{1}{\sqrt{10}} \big (3\ket{0} + \ket{1} \big)$$

This rotation should only be applied if the first qubit is in state $\ket{0}$, that is, you need a conditional-on-zero rotation. The rotation angle can be determined by $\cos\frac{\theta_2}{2} = \frac{3}{\sqrt{10}}$ and $\sin\frac{\theta_2}{2} = \frac{1}{\sqrt{10}}$.

**Step 2.b:** Similarly, the transformation

$$\big (\sqrt{\frac{2}{12}}\ket{1} \big) \otimes \ket{0} \overset{R_{y_3}}\rightarrow \frac{1}{\sqrt{12}} \big (\ket{10}+\ket{11} \big)$$

is equivalent to

$$\ket{1} \otimes \ket{0} \overset{R_{y_3}}\rightarrow \frac{1}{\sqrt{2}} \big (\ket{10}+\ket{11} \big)$$

and can be done using a controlled rotation, applied if first qubit is $\ket{1}$, that is, a conditional-on-one rotation. The rotation angle can be determined by $\cos\frac{\theta_3}{2} = \frac{1}{\sqrt{2}}$ and $\sin\frac{\theta_3}{2} = \frac{1}{\sqrt{2}}$.

In [ ]:
def prepare_hardy_state(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    from math import acos, pi, sqrt
    circ.ry(2 * acos(sqrt(10/12)), qr[0])
    circ.cry(2 * acos(3 / sqrt(10)), qr[0], qr[1], ctrl_state=0)
    circ.cry(2 * pi / 4, qr[0], qr[1])

## Exercise 17. Prepare W state

The solution relies on performing a sequence of controlled rotations.

To prepare a weighted superposition $\cos \theta \ket{0} + \sin \theta \ket{1}$ on a single qubit, you need to start with the $\ket{0}$ state and apply the $R_y$ gate to it with the angle parameter equal to $2 \theta$. 
You'll apply the $R_y$ gate with angle $2 \theta_1 = 2\arcsin \frac{1}{\sqrt{N}}$ to the first qubit of the register to prepare the following state:

$$(\cos \theta_1 \ket{0} + \sin \theta_1 \ket{1}) \otimes \ket{0 \dots 0} = \tfrac{1}{\sqrt{N}}\ket{10 \dots 0} + \tfrac{\sqrt{N-1}}{\sqrt{N}}\ket{00 \dots 0} $$

The first term $\frac{1}{\sqrt{N}}\ket{10 \dots 0}$ already matches the first term of the $\ket{W_N}$ state; now you need to convert the second term $\frac{\sqrt{N-1}}{\sqrt{N}}\ket{00 \dots 0}$ into the rest of the $\ket{W_N}$ terms.

To prepare a term that matches the second term of the $\ket{W_N}$ state, you can apply another $R_y$ gate to the term $\ket{00 \dots 0}$, this time to the second qubit, with an angle $2 \theta_2 = 2\arcsin \frac{1}{\sqrt{N-1}}$.
To make sure it doesn't affect the first term that you're already happy with, you'll apply a controlled version of the $R_y$ gate, with the first qubit of the register in state $\ket{0}$ as control.
This will change the state to

$$\tfrac{1}{\sqrt{N}}\ket{10 \dots 0} + \tfrac{\sqrt{N-1}}{\sqrt{N}} \ket{0} \otimes (\cos \theta_2 \ket{0} + \sin \theta_2 \ket{1}) \otimes \ket{0 \dots 0} = $$
$$= \tfrac{1}{\sqrt{N}}\ket{10 \dots 0} + \tfrac{\sqrt{N-1}}{\sqrt{N}} \tfrac{1}{\sqrt{N-1}} \ket{010 \dots 0} + \tfrac{\sqrt{N-1}}{\sqrt{N}} \tfrac{\sqrt{N-2}}{\sqrt{N-1}} \ket{000 \dots 0}$$

Now you have the first two terms that match the terms of the $\ket{W_N}$ state, and need to convert the third term $\frac{\sqrt{N-2}}{\sqrt{N}}\ket{00 \dots 0}$ into the rest of terms.

You'll keep going like this, preparing one term of the $\ket{W_N}$ state at a time, until the rotation on the last qubit will be an $X$ gate, controlled on all previous $N-1$ qubits being in the $\ket{0 \dots 0}$ state.


In [ ]:
def prepare_wstate(circ: QuantumCircuit, qr: QuantumRegister) -> None:
    from math import asin, sqrt
    from qiskit.circuit.library.standard_gates import RYGate

    n = qr.size
    circ.ry(2 * asin(1 / sqrt(n)), qr[0])
    for ind in range(1, n):
        circ.append(RYGate(2 * asin(1 / sqrt(n - ind))).control(ind, ctrl_state=0), qr[:ind + 1])